# Packages

In [1]:
from __future__ import print_function
import torch.nn as nn
import torchvision.models as models
import torch
import torch.nn as nn
import torch.nn.functional as functional
import torch.optim as optim
from torchvision import transforms
import cv2
import argparse
import time
import torch
import torchvision
import os
import random
import torch.nn.functional as functional

/Users/yunhuinie/opt/anaconda3/envs/tensorflow/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Content loss：

In [2]:
class ContentLoss(nn.Module):
    """
    内容损失层
    """
    def __init__(self, target):
        super(ContentLoss, self).__init__()
        self.target = target.detach()
        self.loss = None

    def forward(self, input):
        self.loss = functional.mse_loss(input, self.target)
        return input

    def update(self, target):
        """
        内容损失的更新函数
        """
        self.target = target.detach()

# Style loss：

In [3]:
class StyleLoss(nn.Module):
    """
    风格损失层
    """
    def __init__(self, target, patch_size, mrf_style_stride, mrf_synthesis_stride, gpu_chunck_size, device):
        super(StyleLoss, self).__init__()
        self.patch_size = patch_size
        self.mrf_style_stride = mrf_style_stride
        self.mrf_synthesis_stride = mrf_synthesis_stride
        self.gpu_chunck_size = gpu_chunck_size
        self.device = device
        self.loss = None

        self.style_patches = self.patches_sampling(target.detach(), patch_size=self.patch_size, stride=self.mrf_style_stride)
        self.style_patches_norm = self.cal_patches_norm()
        self.style_patches_norm = self.style_patches_norm.view(-1, 1, 1)

    def update(self, target):
        """
        更新风格损失的目标函数
        """
        self.style_patches = self.patches_sampling(target.detach(), patch_size=self.patch_size,
                                                   stride=self.mrf_style_stride)
        self.style_patches_norm = self.cal_patches_norm()
        self.style_patches_norm = self.style_patches_norm.view(-1, 1, 1)

    def forward(self, input):
        """
        计算马尔可夫损失，输入参数为合成图像
        """
        synthesis_patches = self.patches_sampling(input, patch_size=self.patch_size, stride=self.mrf_synthesis_stride)
        max_response = []
        for i in range(0, self.style_patches.shape[0], self.gpu_chunck_size):
            i_start = i
            i_end = min(i+self.gpu_chunck_size, self.style_patches.shape[0])
            weight = self.style_patches[i_start:i_end, :, :, :]
            response = functional.conv2d(input, weight, stride=self.mrf_synthesis_stride)
            max_response.append(response.squeeze(dim=0))
        max_response = torch.cat(max_response, dim=0)

        max_response = max_response.div(self.style_patches_norm)
        max_response = torch.argmax(max_response, dim=0)
        max_response = torch.reshape(max_response, (1, -1)).squeeze()
        # 损失
        loss = 0
        for i in range(0, len(max_response), self.gpu_chunck_size):
            i_start = i
            i_end = min(i+self.gpu_chunck_size, len(max_response))
            tp_ind = tuple(range(i_start, i_end))
            sp_ind = max_response[i_start:i_end]
            loss += torch.sum(torch.mean(torch.pow(synthesis_patches[tp_ind, :, :, :]-self.style_patches[sp_ind, :, :, :], 2), dim=[1, 2, 3]))
        self.loss = loss/len(max_response)
        return input

    def patches_sampling(self, image, patch_size, stride):
        """
        对块采样形成一个图片
        第二个参数为图像
        第三个参数为块的大小
        """
        h, w = image.shape[2:4]
        patches = []
        for i in range(0, h - patch_size + 1, stride):
            for j in range(0, w - patch_size + 1, stride):
                patches.append(image[:, :, i:i + patch_size, j:j + patch_size])
        patches = torch.cat(patches, dim=0).to(self.device)
        return patches

    def cal_patches_norm(self):
        """
        计算风格图片块的标准矩阵
        """
        norm_array = torch.zeros(self.style_patches.shape[0])
        for i in range(self.style_patches.shape[0]):
            norm_array[i] = torch.pow(torch.sum(torch.pow(self.style_patches[i], 2)), 0.5)
        return norm_array.to(self.device)

In [4]:
class TVLoss(nn.Module):
    def __init__(self):
        """
        总变差损失层
        """
        super(TVLoss, self).__init__()
        self.loss = None

    def forward(self, input):
        image = input.squeeze().permute([1, 2, 0])
        r = (image[:, :, 0] + 2.12) / 4.37
        g = (image[:, :, 1] + 2.04) / 4.46
        b = (image[:, :, 2] + 1.80) / 4.44

        temp = torch.cat([r.unsqueeze(2), g.unsqueeze(2), b.unsqueeze(2)], dim=2)
        gx = torch.cat((temp[1:, :, :], temp[-1, :, :].unsqueeze(0)), dim=0)
        gx = gx - temp

        gy = torch.cat((temp[:, 1:, :], temp[:, -1, :].unsqueeze(1)), dim=1)
        gy = gy - temp

        self.loss = torch.mean(torch.pow(gx, 2)) + torch.mean(torch.pow(gy, 2))
        return input

# Convolution layer of Markov random matrices

In [5]:
class CNNMRF(nn.Module):
    def __init__(self, style_image, content_image, device, content_weight, style_weight, tv_weight, gpu_chunck_size=256, mrf_style_stride=2,
                 mrf_synthesis_stride=2):
        super(CNNMRF, self).__init__()
        # 在内容和风格之间调解权重
        self.content_weight = content_weight
        self.style_weight = style_weight
        self.tv_weight = tv_weight
        self.patch_size = 3
        self.device = device
        self.gpu_chunck_size = gpu_chunck_size
        self.mrf_style_stride = mrf_style_stride
        self.mrf_synthesis_stride = mrf_synthesis_stride
        self.style_layers = [11, 20]
        self.content_layers = [22]
        self.model, self.content_losses, self.style_losses, self.tv_loss = \
            self.get_model_and_losses(style_image=style_image, content_image=content_image)

    def forward(self, synthesis):
        """
        计算损失并返回新的损失
        参数为合成的图像
        """
        self.model(synthesis)
        style_score = 0
        content_score = 0
        tv_score = self.tv_loss.loss

        # 计算风格损失
        for sl in self.style_losses:
            style_score += sl.loss

        # 计算内容损失
        for cl in self.content_losses:
            content_score += cl.loss

        # 计算最后的损失
        loss = self.style_weight * style_score + self.content_weight * content_score + self.tv_weight * tv_score
        return loss

    def update_style_and_content_image(self, style_image, content_image):
        """
        更新内容的目标损失层和风格的目标损失层
        第一个参数为风格图片
        第二个参数为内容图片
        """
        # 更新风格目标损失层
        x = style_image.clone()
        next_style_idx = 0
        i = 0
        for layer in self.model:
            if isinstance(layer, TVLoss) or isinstance(layer, ContentLoss) or isinstance(layer, StyleLoss):
                continue
            if next_style_idx >= len(self.style_losses):
                break
            x = layer(x)
            if i in self.style_layers:
                # 在vgg模型中提取风格特征作为风格损失目标
                self.style_losses[next_style_idx].update(x)
                next_style_idx += 1
            i += 1

        # 更新内容损失层目标函数
        x = content_image.clone()
        next_content_idx = 0
        i = 0
        for layer in self.model:
            if isinstance(layer, TVLoss) or isinstance(layer, ContentLoss) or isinstance(layer, StyleLoss):
                continue
            if next_content_idx >= len(self.content_losses):
                break
            x = layer(x)
            if i in self.content_layers:
                # 在vgg模型中提取内容特征作为内容损失目标
                self.content_losses[next_content_idx].update(x)
                next_content_idx += 1
            i += 1

    def get_model_and_losses(self, style_image, content_image):
        """
        通过插入vgg19和自定义的损失函数层创建网络模型
        第一个参数为风格图片
        第二个参数为内容图片
        """
        vgg = models.vgg19(pretrained=True).to(self.device)
        model = nn.Sequential()
        content_losses = []
        style_losses = []
        # 添加总变差损失层
        tv_loss = TVLoss()
        model.add_module('tv_loss', tv_loss)

        next_content_idx = 0
        next_style_idx = 0

        for i in range(len(vgg.features)):
            if next_content_idx >= len(self.content_layers) and next_style_idx >= len(self.style_layers):
                break
            # 加入vgg19模型
            layer = vgg.features[i]
            name = str(i)
            model.add_module(name, layer)

            # 加入内容损失层
            if i in self.content_layers:
                target = model(content_image).detach()
                content_loss = ContentLoss(target)
                model.add_module("content_loss_{}".format(next_content_idx), content_loss)
                content_losses.append(content_loss)
                next_content_idx += 1

            # 加入风格损失层
            if i in self.style_layers:
                target_feature = model(style_image).detach()
                style_loss = StyleLoss(target_feature, patch_size=self.patch_size, mrf_style_stride=self.mrf_style_stride,
                                       mrf_synthesis_stride=self.mrf_synthesis_stride, gpu_chunck_size=self.gpu_chunck_size, device=self.device)

                model.add_module("style_loss_{}".format(next_style_idx), style_loss)
                style_losses.append(style_loss)
                next_style_idx += 1

        return model, content_losses, style_losses, tv_loss

In [6]:
def get_synthesis_image(synthesis, denorm, device):
    """
    获取合成图片，去规范化：从tensor转为numpy矩阵形式
    第一个参数为合成图像的tensor形式
    第二个参数是去规范化的方法
    """
    cpu_device = torch.device('cpu')

    image = synthesis.clone().squeeze().to(cpu_device)

    image = denorm(image)  #图片去规范化，还原

    return image.to(device).clamp_(0, 1)

In [7]:
def unsample_synthesis(height, width, synthesis, device):
    """
    对合成图片进行下采样
    第一个参数为下采样图像的高度
    第二个参数为下采样图像的宽度
    第三个参数为合成图像准备下采样的tensor形式
    第四个参数表示使用的是cpu还是gpu
    """
    # 将tensor转换为numpy形式，并作为图片进行下采样
    synthesis = functional.interpolate(synthesis, size=[height, width], mode='bilinear')  #双线值插值方法
    synthesis = synthesis.clone().detach().requires_grad_(True).to(device)
    return synthesis

In [16]:
def main(content_path,style_path,max_iter,sample_step,content_weight,style_weight,tv_weight,num_res,gpu_chunck_size,mrf_style_stride,mrf_synthesis_stride):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    "转换与去规范化转换"
    # 预训练的vgg网络，其图片的均值为[0.485, 0.456, 0.406]，方差为[0.229, 0.224, 0.225].
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))])
    denorm_transform = transforms.Normalize(mean=(-2.12, -2.04, -1.80), std=(4.37, 4.46, 4.44))

    "读取图片"
    if not os.path.exists(content_path):
        raise ValueError('file %s does not exist.' % content_path)
    if not os.path.exists(style_path):
        raise ValueError('file %s does not exist.' % style_path)
    content_image = cv2.imread(content_path)
    content_image = cv2.cvtColor(content_image, cv2.COLOR_BGR2RGB)
    content_image = transform(content_image).unsqueeze(0).to(device)

    style_image = cv2.imread(style_path)
    style_image = cv2.cvtColor(style_image, cv2.COLOR_BGR2RGB)
    style_image = transform(style_image).unsqueeze(0).to(device)

    "重新设置图片的大小"
    pyramid_content_image = []
    pyramid_style_image = []
    for i in range(num_res):
        content = functional.interpolate(content_image, scale_factor=1/pow(2, num_res-1-i), mode='bilinear')

        style = functional.interpolate(style_image, scale_factor=1/pow(2, num_res-1-i), mode='bilinear')

        pyramid_content_image.append(content)
        pyramid_style_image.append(style)
    "开始训练"
    global iter
    iter = 0
    synthesis = None
    # 创建卷积马尔可夫随机阵模型
    cnnmrf = CNNMRF(style_image=pyramid_style_image[0], content_image=pyramid_content_image[0], device=device,
                    content_weight=content_weight, style_weight=style_weight, tv_weight=tv_weight,
                    gpu_chunck_size=gpu_chunck_size, mrf_synthesis_stride=mrf_synthesis_stride,
                    mrf_style_stride=mrf_style_stride).to(device)

    # 设置模块进入训练模式
    cnnmrf.train()
    for i in range(0, num_res):
        # synthesis = torch.rand_like(content_image, requires_grad=True)
        if i == 0:
            # 低维度用于合成图像中内容图像特征提取
            synthesis = pyramid_content_image[0].clone().requires_grad_(True).to(device)
        else:
            # 高维度将合成图像从更高层进行下采样
            synthesis = unsample_synthesis(pyramid_content_image[i].shape[2], pyramid_content_image[i].shape[3], synthesis, device)
            cnnmrf.update_style_and_content_image(style_image=pyramid_style_image[i], content_image=pyramid_content_image[i])
        # 设置优化迭代的最大次数
        optimizer = optim.LBFGS([synthesis], lr=1, max_iter=max_iter)
        "--------------------"

        def closure():
            global iter
            optimizer.zero_grad()
            loss = cnnmrf(synthesis)
            loss.backward(retain_graph=True)
            # 显示出每次优化的损失值
            if (iter + 1) % 10 == 0:
                print('res-%d-iteration-%d: %f' % (i+1, iter + 1, loss.item()))
            # 保存图片
            if (iter + 1) % sample_step == 0 or iter + 1 == max_iter:
                image = get_synthesis_image(synthesis, denorm_transform, device)
                image = functional.interpolate(image.unsqueeze(0), size=content_image.shape[2:4], mode='bilinear')
                torchvision.utils.save_image(image.squeeze(), './Li_Style_Transfer_results/results/res-%d-result-%d.jpg' % (i+1, iter + 1))
                print('save image: res-%d-result-%d.jpg' % (i+1, iter + 1))
            iter += 1
            if iter == max_iter:
                iter = 0
            return loss

        "----------------------"
        optimizer.step(closure)


if __name__ == '__main__':

    mylist = os.listdir('./Test_dataset')
    root_path = './Test_dataset/'

#     random.seed(1)
#     content_image_path = root_path + random.choice(mylist) 
    content_image_path = root_path + '000000002897.jpg'
    style_image_path = './Alexnet_Style_Transfer_results/style_input/Kazimir.jpg'  #指定一个风格图片

    start_time = time.time()
    main(content_image_path,       #内容图片的路径
         style_image_path,    #风格图片的路径
             200,        #最大迭代次数
             50,         #每多少步采样
             1,          #内容图像损失函数权重
             0.4,        #风格图像损失函数权重
             0.1,        #总变差损失函数权重
             3,          #循环次数
             512,        #gpu块大小
             2,          #马尔科夫随机阵风格步长
             2)          #马尔科夫随机阵合成步长
    end_time = time.time()
    elapsed_time = time.strftime("%H:%M:%S", time.gmtime(end_time - start_time))

    print("模型训练总时长为: %s" % elapsed_time)

/Users/yunhuinie/opt/anaconda3/envs/tensorflow/lib/python3.6/site-packages/torch/nn/functional.py:3635: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)
/Users/yunhuinie/opt/anaconda3/envs/tensorflow/lib/python3.6/site-packages/torch/nn/functional.py:3680: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and now uses scale_factor directly, instead of relying on the computed output size. If you wish to restore the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  "The default behavior for interpolate/upsample with float scale_factor changed "


res-1-iteration-10: 43.044933
res-1-iteration-20: 42.109062
res-1-iteration-30: 41.510006
res-1-iteration-40: 41.014008
res-1-iteration-50: 40.609325
save image: res-1-result-50.jpg
res-1-iteration-60: 40.344543
res-1-iteration-70: 40.220802
res-1-iteration-80: 40.288956
res-1-iteration-90: 40.091732
res-1-iteration-100: 40.072079
save image: res-1-result-100.jpg
res-1-iteration-110: 39.936611
res-1-iteration-120: 39.840050
res-1-iteration-130: 39.803291
res-1-iteration-140: 39.722916
res-1-iteration-150: 39.675804
save image: res-1-result-150.jpg
res-1-iteration-160: 39.946712
res-1-iteration-170: 39.842201
res-1-iteration-180: 39.854626
res-1-iteration-190: 39.919727
res-1-iteration-200: 39.736038
save image: res-1-result-200.jpg
res-2-iteration-10: 30.737274
res-2-iteration-20: 27.304382
res-2-iteration-30: 26.066414
res-2-iteration-40: 25.464260
res-2-iteration-50: 25.124203
save image: res-2-result-50.jpg
res-2-iteration-60: 24.881317
res-2-iteration-70: 24.708612
res-2-iteration-

In [17]:
import cv2  
import matplotlib.pyplot as plt
img = cv2.imread('./Li_Style_Transfer_results/results/Francis/res-3-result-200.jpg')
# b,g,r = cv2.split(img) 
# img_rgb = cv2.merge([r,g,b]) 
# plt.figure() 
# plt.imshow(img_rgb) 
# plt.show() 
cv2.imwrite('./4.jpg',cv2.cvtColor(img,cv2.COLOR_BGR2RGB))



True